In [1]:
from os import listdir
from os.path import isfile, join
import plotly.graph_objects as go
import networkx as nx
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html

from matplotlib import pylab
import matplotlib.pyplot as plt
import networkx as nx
import csv
import webbrowser

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy c-extensions failed.
- Try uninstalling and reinstalling numpy.
- If you have already done that, then:
  1. Check that you expected to use Python3.7 from "C:\ProgramData\Anaconda3\envs\X5DB\python.exe",
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy version "1.18.1" you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed numpy
     - your operating system
     - whether or not you have multiple versions of Python installed
     - if you built from source, your compiler versions and ideally a build log

- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.

Note: this error has many possible c

In [2]:
# get all duplicate results
docs = eval(open('final_out_dupe_detect.txt', 'r').read()) # duplicate result array
final_docs = [[i[0], list(set(i[2]) & set(i[3]))] for i in docs] # array with material ids and its duplicates

In [3]:
f=open("out_duplicates.txt","r")
final_docs = eval(f.read())
print("documents count : "+str(len(final_docs)))

# get urls and word counts for all oer materials
reader=csv.reader(open("./urls.csv", newline=''))
urls = [a for a in reader]

word_count = {int(i[2]):int(i[0]) for i in urls}
url_dict = {int(i[2]):i[1] for i in urls}

# initializing the graph
G = nx.Graph()
duplicate_count = 2 # change this value for the graph to show documents with more than this number of duplicates

# all the nodes and edges for duplicate pairs
nodes = list(set([i[0] for i in final_docs if len(i[1]) > duplicate_count] + [j for i in final_docs for j in i[1] if len(i[1]) > duplicate_count]))
edges = []
for i in final_docs:
    for j in i[1]:
        if i[0] != j and len(i[1])>duplicate_count:
            edges.append((i[0], j))
G.add_nodes_from(nodes)
for e in edges:
    G.add_edge(*e)

# Graph info
print(nx.info(G))



FileNotFoundError: [Errno 2] No such file or directory: 'out_duplicates.txt'

In [11]:
# get graph positions for each node
pos=nx.spring_layout(G)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

# defining all the edges
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

# defining all the nodes
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

# add hover labels : material ids, no of duplicates, word_count
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    if adjacencies[0] in word_count:
        node_text.append('material_id = '+str(adjacencies[0])+'<br>No of connections = '+str(len(adjacencies[1]))+"<br>word_count = "+str(word_count[adjacencies[0]]))
    else:
        node_text.append('material_id = '+str(adjacencies[0])+'<br>No of connections = '+str(len(adjacencies[1])))        
node_trace.marker.color = node_adjacencies
node_trace.text = node_text

fig = go.FigureWidget(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Duplicates Graph',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=True))
                )

node=[i for i in G.nodes()]
# call back for the click event on points to direct to oer materials
def update_point(trace, points, selector):
    try:
        webbrowser.open(url_dict[node[points.point_inds[0]]])
    except Exception as e:
        print(e)
fig.data[1].on_click(update_point)

fig

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': '#888', 'width': 0.5},
     …

In [12]:
reverse_url=dict([(value, key) for key, value in url_dict.items()])
len(word_count)
all_d = [i+[word_count[i[0]]] if i[0] in word_count else i for i in final_docs]
len(all_d)

112136

In [25]:
len([i for i in all_d if len(i[1])>1 and len(i)>2 and i[2]<100])

293